In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Load the final dataset from Google Drive
final_data_path = "/content/drive/My Drive/final_data.csv"
final_data = pd.read_csv(final_data_path)

# Display the first few rows to verify
print("Final dataset preview:")
print(final_data.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import numpy as np
import os

# Ensure date column is in datetime format
final_data['date'] = pd.to_datetime(final_data['date'], errors='coerce')

# Ensure necessary columns exist
required_cols = ['Sex_speakerinfo', 'Commission_speakerinfo', 'date', 'w_top_tot', 'w_top_rel']
for col in required_cols:
    if col not in final_data.columns:
        raise KeyError(f"Column '{col}' is missing from the dataset.")

# **Aggregate values per quarter, sex, and commission**
final_data['year_quarter'] = final_data['date'].dt.to_period("Q")  # Convert to quarterly periods

aggregated_data = final_data.groupby(['year_quarter', 'Commission_speakerinfo', 'Sex_speakerinfo'])[['w_top_tot', 'w_top_rel']].mean().reset_index()

# Convert year-quarter to datetime for plotting (middle of each quarter for proper ordering)
aggregated_data['year_quarter'] = aggregated_data['year_quarter'].astype(str)
aggregated_data['date'] = pd.to_datetime(aggregated_data['year_quarter'].apply(lambda x: f"{x[:4]}-{int(x[-1])*3-1}-15"))

# **Create separate plots for each Commission_speakerinfo**
commissions = aggregated_data['Commission_speakerinfo'].unique()

# **Set up multiple plots**
fig, axes = plt.subplots(len(commissions), 1, figsize=(12, len(commissions) * 4), sharex=True)

if len(commissions) == 1:
    axes = [axes]  # Ensure axes is iterable for a single Commission case

for ax, commission in zip(axes, commissions):
    data_subset = aggregated_data[aggregated_data['Commission_speakerinfo'] == commission]

    # Split by Sex
    data_men = data_subset[data_subset['Sex_speakerinfo'] == 0]
    data_women = data_subset[data_subset['Sex_speakerinfo'] == 1]

    # **Plot Women's Interest Percentage Over Time**
    ax.plot(data_men['date'], data_men['w_top_tot'], color="#003399", linestyle='-', linewidth=2, label="Women’s Issues (Men)")
    ax.plot(data_women['date'], data_women['w_top_tot'], color="#FFCC00", linestyle='-', linewidth=2, label="Women’s Issues (Women)")

    # **Plot Relative Women's Interest Over Time**
    ax.plot(data_men['date'], data_men['w_top_rel'], color="#003399", linestyle='--', linewidth=2, label="Relative Women’s Issues (Men)")
    ax.plot(data_women['date'], data_women['w_top_rel'], color="#FFCC00", linestyle='--', linewidth=2, label="Relative Women’s Issues (Women)")

    # Formatting
    ax.set_title(f"Women's Interest Over Time ({commission})")
    ax.set_ylabel("Percentage (%)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)

# **Format x-axis for continuous time display**
plt.xlabel("Year-Quarter")
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

plt.tight_layout()

# **Save the plot as an SVG vector graphic**
save_folder = "/content/drive/My Drive/phd plots"
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder, "women_interest_by_commission_quarters.svg")
plt.savefig(save_path, format="svg")

# Show the plot
plt.show()

print(f"Plot saved as vector graphic at: {save_path}")


In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# Ensure data is clean
final_data = final_data.dropna(subset=['claimbinary', 'claimcount', 'length', 'Sex_speakerinfo', 'Commission_speakerinfo', 'speaker.name'])

# Convert categorical variables
final_data['Commission_speakerinfo'] = final_data['Commission_speakerinfo'].astype("category")
final_data['speaker.name'] = final_data['speaker.name'].astype("category")

logit_model = smf.mixedlm("claimbinary ~ length + Sex_speakerinfo",
                          final_data,
                          groups=final_data["speaker.name"],  # Speaker-level random effects
                          re_formula="1")  # Random intercept

logit_result = logit_model.fit()

print("\n🚀 Logistic Mixed Model Results (Claim Probability):\n")
print(logit_result.summary())

poisson_model = smf.mixedlm("claimcount ~ length + Sex_speakerinfo",
                             final_data,
                             groups=final_data["speaker.name"],
                             re_formula="1")  # Random intercept

poisson_result = poisson_model.fit()

print("\n🚀 Poisson Mixed Model Results (Claim Count):\n")
print(poisson_result.summary())

mean_claims = final_data['claimcount'].mean()
var_claims = final_data['claimcount'].var()

print(f"\n📊 Mean of claimcount: {mean_claims:.2f}")
print(f"📊 Variance of claimcount: {var_claims:.2f}")

if var_claims > mean_claims * 1.5:
    print("\n⚠️ Overdispersion detected! Using Negative Binomial instead of Poisson.")

    nb_model = smf.mixedlm("claimcount ~ length + Sex_speakerinfo",
                           final_data,
                           groups=final_data["speaker.name"],
                           re_formula="1")  # Random intercept

    nb_result = nb_model.fit()
    print("\nNegative Binomial Mixed Model Results (Claim Count):\n")
    print(nb_result.summary())


We need a negative binominal to analyse the data -> switch to R and use brms/glmmTMB. Code is provided on GitHub.